# PuLPモジュールを用いたナップサック問題の最適解の計算
## PuLPモジュール読み込み

In [1]:
import pulp

## 目的関数の係数および制約条件左辺の係数を定義
変数が多いので，効率的な記述のため，目的関数の係数および制約条件左辺の係数を辞書として定義する．
辞書はキーと値の組の集まりで，{key1:value1, key2:value2, key3:value3}のように書く．
キーにより要素の値を取りだしたり，代入したりすることができる．
例えば，    
    
    d1 = {3:4, 'a':5, 2:'abc'}
という辞書を作ったとする．
d1の要素の値は，d1[3], d1['a'], d1[2]により得ることができ，各々4, 5, 'abc'を得る．
複数の要素をまとめたデータ型としては，リストやタプルというデータ型があるが，これらにおける要素へのアクセスはキーではなくく，要素の並び順であるインデックスにより行う．
リストやタプルの先頭の要素のインデックスは0なので，「品物1から品物5」とは番号がずれる．
ここでは，可能な限り問題を素直に定式化していくために，辞書を採用する．
Cはキーが品物番号，値が品物の価値の辞書，Aはキーが品物番号，値が重さの辞書とする．

In [2]:
C = {1:9, 2:7, 3:6, 4:5, 5:3} # 価値
A = {1:6, 2:4, 3:5, 4:3, 5:3} # 重さ

## 決定変数の定義
決定変数x1～x5も{1:x1, 2:x2, 3:x3, 4:x4, 5:x5}という辞書にする．変数は，
    
    pulp.LpVariable(f'x{i+1}', cat=pulp.LpBinary)
という形をとる．
変数名はf-stringでf'x{i}'とすることで，品物の番号に相当する変数iが展開され，x1, x2, ..., x5となるようにする．
ナップサック問題の決定変数は0/1の2値変数なので，cat=pulp.LpBinaryとする．品物の番号をキー，値をその品物をナップサックに詰めるかいなかを表す決定変数とする辞書は次のように作る．   

In [3]:
x ={i:pulp.LpVariable(f'x{i}', cat=pulp.LpBinary) for i in C.keys()}

{}内の前の部分は i:pulp.LpVariable() でキーと値の組になっている．後の部分は変数iに品物番号1～5を順番に代入している．C.keys()は先に定義した辞書Cのキーをすべて取り出す．このように定義することによりxは以下のようになる．

In [4]:
x

{1: x1, 2: x2, 3: x3, 4: x4, 5: x5}

## 問題の定義
pulp.LpProblemで問題オブジェクトを生成した後，目的関数と制約条件を設定する．
目的関数の設定における

    pulp.lpSum(C[i]*x[i] for i in x.keys())
は，C[i]*x[i]のi=1からi=5までの和を表していて，$\sum_{i=1}^{5}c_i x_i$に相当する．
制約条件もpulp.lpSum()を用いて同様に書くことができる．

In [5]:
p = pulp.LpProblem('ナップサック問題', sense=pulp.LpMaximize)
p += pulp.lpSum(C[i]*x[i] for i in x.keys()), '目的関数　詰めた品物の価値'
p += pulp.lpSum(A[i]*x[i] for i in x.keys()) <= 17, '重量制約'
p

ナップサック問題:
MAXIMIZE
9*x1 + 7*x2 + 6*x3 + 5*x4 + 3*x5 + 0
SUBJECT TO
重量制約: 6 x1 + 4 x2 + 5 x3 + 3 x4 + 3 x5 <= 17

VARIABLES
0 <= x1 <= 1 Integer
0 <= x2 <= 1 Integer
0 <= x3 <= 1 Integer
0 <= x4 <= 1 Integer
0 <= x5 <= 1 Integer

## 最適解の計算と結果の読み取り

In [6]:
result = p.solve()

In [7]:
pulp.LpStatus[result]

'Optimal'

In [8]:
pulp.value(p.objective)

24.0

In [9]:
for v in p.variables():
    print(f'{v} = {pulp.value(v):.0f}')

x1 = 1
x2 = 1
x3 = 0
x4 = 1
x5 = 1


{pulp.value(v):.0f}の :.0f は小数点以下を表示しないようにする．
最適解は，x1 = x2 = x4 = x5 =1, x3 = 0で，品物1, 2, 4, 5をナップサックに詰め，品物3をナップサックに詰め込まないのが，ナップサック内の品物の価値の合計を最大にし，その値は24である．